In [1]:
import pandas as pd
import numpy as np

In [2]:
# read csv files
df_AQI_2023_01 = pd.read_csv('AQI-2023-01.csv')
df_AQI_2023_02 = pd.read_csv('AQI-2023-02.csv')
df_AQI_2023_03 = pd.read_csv('AQI-2023-03.csv')

df_AQI_2023_01.info()
print()
df_AQI_2023_02.info()
print()
df_AQI_2023_03.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63984 entries, 0 to 63983
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   "sitename"          63984 non-null  object 
 1   "county"            63984 non-null  object 
 2   "aqi"               63984 non-null  int64  
 3   "pollutant"         38178 non-null  object 
 4   "status"            63946 non-null  object 
 5   "so2"               62783 non-null  float64
 6   "co"                63256 non-null  float64
 7   "o3"                63193 non-null  float64
 8   "o3_8hr"            63724 non-null  float64
 9   "pm10"              63246 non-null  float64
 10  "pm2.5"             63307 non-null  float64
 11  "no2"               62891 non-null  float64
 12  "nox"               62891 non-null  float64
 13  "no"                62891 non-null  float64
 14  "windspeed"         63791 non-null  object 
 15  "winddirec"         63789 non-null  object 
 16  "dat

In [3]:
# assign specific keys
keys = ['"siteid"', '"sitename"','"aqi"', '"pm2.5_avg"', '"pm10_avg"', '"so2_avg"', '"datacreationdate"']

# select data in 高雄市
df_AQI_2023_01_specific = df_AQI_2023_01[df_AQI_2023_01['"county"'] == "高雄市"][keys]
df_AQI_2023_02_specific = df_AQI_2023_02[df_AQI_2023_02['"county"'] == "高雄市"][keys]
df_AQI_2023_03_specific = df_AQI_2023_03[df_AQI_2023_03['"county"'] == "高雄市"][keys]

df_AQI_2023_01_specific.info()
print()
df_AQI_2023_02_specific.info()
print()
df_AQI_2023_03_specific.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9672 entries, 3 to 63977
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   "siteid"            9672 non-null   int64  
 1   "sitename"          9672 non-null   object 
 2   "aqi"               9672 non-null   int64  
 3   "pm2.5_avg"         9672 non-null   float64
 4   "pm10_avg"          9670 non-null   float64
 5   "so2_avg"           9653 non-null   float64
 6   "datacreationdate"  9672 non-null   object 
dtypes: float64(3), int64(2), object(2)
memory usage: 604.5+ KB

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8736 entries, 9 to 57248
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   "siteid"            8736 non-null   int64  
 1   "sitename"          8736 non-null   object 
 2   "aqi"               8736 non-null   float64
 3   "pm2.5_avg"         8734 non-n

In [4]:
# combine AQI dfs
df_AQI = pd.concat([df_AQI_2023_01_specific, \
                    df_AQI_2023_02_specific, \
                    df_AQI_2023_03_specific], \
                    axis=0, ignore_index=True)

df_AQI.dropna(inplace=True)
df_AQI.reset_index(inplace=True)
df_AQI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27910 entries, 0 to 27909
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               27910 non-null  int64  
 1   "siteid"            27910 non-null  int64  
 2   "sitename"          27910 non-null  object 
 3   "aqi"               27910 non-null  float64
 4   "pm2.5_avg"         27910 non-null  float64
 5   "pm10_avg"          27910 non-null  float64
 6   "so2_avg"           27910 non-null  float64
 7   "datacreationdate"  27910 non-null  object 
dtypes: float64(4), int64(2), object(2)
memory usage: 1.7+ MB


In [5]:
# transfer date (YYYY-MM-DD HH:MM) -> (YYYY-MM-DD)
print("Before:")
print(df_AQI['"datacreationdate"'])

df_AQI['"datacreationdate"'] = pd.to_datetime(df_AQI['"datacreationdate"'])
df_AQI['"datacreationdate"'] = df_AQI['"datacreationdate"'].dt.date
    
print("\nAfter:")
print(df_AQI['"datacreationdate"'])

Before:
0        2023-01-01 00:00
1        2023-01-01 00:00
2        2023-01-01 00:00
3        2023-01-01 00:00
4        2023-01-01 00:00
               ...       
27905    2023-03-31 23:00
27906    2023-03-31 23:00
27907    2023-03-31 23:00
27908    2023-03-31 23:00
27909    2023-03-31 23:00
Name: "datacreationdate", Length: 27910, dtype: object

After:
0        2023-01-01
1        2023-01-01
2        2023-01-01
3        2023-01-01
4        2023-01-01
            ...    
27905    2023-03-31
27906    2023-03-31
27907    2023-03-31
27908    2023-03-31
27909    2023-03-31
Name: "datacreationdate", Length: 27910, dtype: object


In [6]:
# create df_result
df_AQI_result = pd.DataFrame(columns=keys)

site_group = df_AQI.groupby(['"siteid"', '"sitename"', '"datacreationdate"'])
for name, group in site_group:
    # get value
    siteid = group.reset_index().loc[0, '"siteid"']
    sitename = group.reset_index().loc[0, '"sitename"']
    datacreationdate = group.reset_index().loc[0, '"datacreationdate"']
    
    # calculate mean
    mean_aqi = round(group['"aqi"'].mean(), 2)
    mean_pm2 = round(group['"pm2.5_avg"'].mean(), 2)
    mean_pm10 = round(group['"pm10_avg"'].mean(), 2)
    mean_so2 = round(group['"so2_avg"'].mean(), 2)
    
    # append to the df_result
    df_AQI_result.loc[len(df_AQI_result)] = [siteid, sitename, mean_aqi, mean_pm2, mean_pm10, mean_so2, datacreationdate]

# output
print(df_AQI_result)
df_AQI_result.to_csv('AQI.csv', index=False)

      "siteid" "sitename"  "aqi"  "pm2.5_avg"  "pm10_avg"  "so2_avg"  \
0           47         美濃  80.83        27.58       45.88       1.00   
1           47         美濃  63.33        20.71       32.54       0.38   
2           47         美濃  77.59        26.27       36.64       0.00   
3           47         美濃  51.25        16.17       27.33       0.00   
4           47         美濃  77.25        26.12       46.08       0.00   
...        ...        ...    ...          ...         ...        ...   
1165       202     高雄(湖內)  76.17        25.62       42.62       1.12   
1166       202     高雄(湖內)  75.46        25.46       41.21       1.00   
1167       202     高雄(湖內)  78.92        26.83       42.88       1.00   
1168       202     高雄(湖內)  68.83        22.75       40.88       1.00   
1169       202     高雄(湖內)  82.54        28.29       47.96       1.21   

     "datacreationdate"  
0            2023-01-01  
1            2023-01-02  
2            2023-01-03  
3            2023-01-04  
4    